In [1]:
import pandas as pd
import MetaTrader5 as mt5
from datetime import datetime,timedelta
import time
import talib as ta
import pytz

In [2]:
ativo = 'EURUSD'
#ativos = ['EURUSD']



path = r'C:\Program Files\MT5 by FOREX.com Terminal\terminal64.exe'

if not mt5.initialize(path = path):
    print("initialize() failed")
    mt5.shutdown()

In [7]:
ativo = 'EURUSD'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M5, 0, 200))
df.columns = ['Data','Open','High','Low','Close','Vol','','']
time_fix = 10800
df['Data'] = df['Data'].apply(lambda x: str(datetime.fromtimestamp(x + time_fix)))
df['MA_9'] = ta.EMA(df['Close'], timeperiod = 9)
df['MA_9_diff_value'] = df['MA_9'].diff(periods = 1)
df['MA_9_tend'] = df['MA_9_diff_value'].apply(lambda x: 1 if x > 0 else -1)
df['MA_9_tend_diff'] = df['MA_9_tend'].diff()
df['MA_100'] = ta.SMA(df['Close'],timeperiod=100)
df['tend'] = df.apply(lambda x: 1 if x['Close'] > x['MA_100'] else 0, axis = 1)

df['acao'] = df.apply(lambda x: 'sell' if (x['MA_9_tend_diff'] == 2 and x['tend'] == 1) 
    else 'call' if  (x['MA_9_tend_diff'] == -2 and x['tend'] == 0)  else 0, axis =1)

df['preco'] = df.apply(lambda x: x['Close'], axis =1)

df['stop'] = df.apply(lambda x: x['High'] + (x['High'] - x['Low'])  if x['acao']=='sell' else x['Low'] - (x['High'] - x['Low']) if x['acao'] == 'call' else 0, axis =1)



##Excluir seguidos
lista = df['acao'].values

tempo_lista = 10
for num, i in enumerate(lista):
    #print(num,i)
    if i != 0:
        for i in range(tempo_lista):
            try:
                n_num = num + i + 1
                lista[n_num] = 0
            except:
                pass

df['acao'] = lista


In [10]:
df.tail(50)

,Data,Open,High,Low,Close,Vol,,,MA_9,MA_9_diff_value,MA_9_tend,MA_9_tend_diff,MA_100,tend,acao,preco,stop
150,2022-12-28 14:25:00,1.06417,1.06468,1.06404,1.06468,299,16,0,1.064227,0.000113,1,0.0,1.064134,1,0,1.06468,0.00000
151,2022-12-28 14:30:00,1.06468,1.06515,1.06461,1.06467,320,15,0,1.064315,0.000089,1,0.0,1.064141,1,0,1.06467,0.00000
152,2022-12-28 14:35:00,1.06468,1.06508,1.06464,1.06474,274,16,0,1.064400,0.000085,1,0.0,1.064151,1,0,1.06474,0.00000
153,2022-12-28 14:40:00,1.06475,1.06536,1.06464,1.06519,311,15,0,1.064558,0.000158,1,0.0,1.064166,1,0,1.06519,0.00000
154,2022-12-28 14:45:00,1.06521,1.06538,1.06477,1.06508,385,16,0,1.064663,0.000104,1,0.0,1.064176,1,0,1.06508,0.00000
155,2022-12-28 14:50:00,1.06508,1.06566,1.06492,1.06563,197,15,0,1.064856,0.000193,1,0.0,1.064195,1,0,1.06563,0.00000
156,2022-12-28 14:55:00,1.06563,1.06565,1.06450,1.06451,374,15,0,1.064787,-0.000069,-1,-2.0,1.064205,1,0,1.06451,0.00000
157,2022-12-28 15:00:00,1.06453,1.06488,1.06453,1.06484,324,15,0,1.064798,0.000011,1,2.0,1.064214,1,sell,1.06484,1.06523
158,2022-12-28 15:05:00,1.06484,1.06485,1.06413,1.06424,304,15,0,1.064686,-0.000112,-1,-2.0,1.064215,1,0,1.06424,0.00000
159,2022-12-28 15:10:00,1.06426,1.06430,1.06373,1.06389,331,16,0,1.064527,-0.000159,-1,0.0,1.064212,0,0,1.06389,0.00000
